### 1. Импорт и загрузка данных 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# фиксируем device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),               # загрузит в [0,1], shape=(1,28,28)
])
train_ds = datasets.MNIST("../data", train=True, download=True, transform=transform)
test_ds  = datasets.MNIST("../data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=1000, shuffle=False)

### 2. Определение модели:

In [ ]:
class MNISTNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding=2)  # 1×28×28 → 16×28×28
        self.pool  = nn.MaxPool2d(2,2)                          # 16×28×28 → 16×14×14
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding=2) # 16×14×14 → 32×14×14
        #→ pool → 32×7×7, flatten → 32*7*7=1568
        self.fc1   = nn.Linear(32*7*7, 128)
        self.fc2   = nn.Linear(128, 10)
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = MNISTNet().to(device)
print(model)

MNISTNet(
  (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=1568, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


### 3. Тренировка:

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(1, 6):   # 5 эпох достаточно для теста
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}, loss={total_loss/len(train_loader):.4f}")

Epoch 1, loss=0.1861
Epoch 2, loss=0.0550
Epoch 3, loss=0.0375
Epoch 4, loss=0.0288
Epoch 5, loss=0.0215


### 4. Проверк точности на тестовой выборке

In [ ]:
model.eval()
correct = 0
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        pred = model(xb).argmax(dim=1)
        correct += (pred == yb).sum().item()
print("Test accuracy:", correct/len(test_ds))

Test accuracy: 0.9894


### 5. Экспорт в ONNX

In [13]:
# Ячейка 6: экспорт в ONNX
dummy = torch.randn(1,1,28,28, device=device)
torch.onnx.export(
    model, dummy, "../model/model.onnx",
    input_names=["input"], output_names=["output"],
    dynamic_axes={"input":{0:"batch"}, "output":{0:"batch"}},
    opset_version=11
)
print("ONNX saved to ../model/model.onnx")

ONNX saved to ../model/model.onnx


# Локальная проверка ONNX


In [19]:
import onnxruntime as rt
import numpy as np
from PIL import Image

# 1) Загрузить сессию ONNX
sess = rt.InferenceSession("../model/model.onnx")
input_name  = sess.get_inputs()[0].name
output_name = sess.get_outputs()[0].name

# 2) Загрузить и подготовить изображение
img = Image.open("../data/sample2.bmp").convert("L")       # grayscale
arr = np.array(img, dtype=np.float32) / 255.0           # нормировать в [0,1]
# ONNX-модель ожидает input shape = (batch, channel, height, width)
input_tensor = arr[np.newaxis, np.newaxis, :, :]

# 3) Запустить инференс
output = sess.run([output_name], {input_name: input_tensor})[0]

# 4) Вывести результаты
print("ONNX output:", output)


ONNX output: [[  1.2481185   -0.46793416  15.569992    -5.461108   -10.614124
  -16.906298    -1.2405177   -5.5630975   -1.717217   -13.229018  ]]
